# Using only the neural network output from this file

In [1]:
##Setting up basic libraries
import numpy as np
import pandas as pd
import random
import datetime as dt
import gc
# Visualization libaries
import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()
%matplotlib inline
import warnings
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn #ignore annoying warning (from sklearn and seaborn)
#Statistical libraries
from scipy import stats
from scipy.stats import norm, skew 
#sklearn for pre-processing the data and prepping test data
from sklearn import preprocessing
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import auc
from sklearn.model_selection import StratifiedKFold,KFold
from sklearn.metrics import roc_auc_score, precision_score, recall_score

from lightgbm import LGBMClassifier
import lightgbm as lgb
from scipy.stats import ranksums
from bayes_opt import BayesianOptimization


In [2]:
dataloc = '/home/sandip/Desktop/AV_Hackathon/April19/input/'
outloc = '/home/sandip/Desktop/AV_Hackathon/April19/output/'

In [3]:
train = pd.read_csv(dataloc + "train.csv")
test = pd.read_csv(dataloc + "test.csv")
train.shape,test.shape

((233154, 41), (112392, 40))

In [4]:
def check_nmiss(df):
    df_na = (df.isnull().sum()/len(df))*100
    df_na = df_na.drop(df_na[df_na == 0].index).sort_values(ascending=False)[:30]
    missing_data = pd.DataFrame({'Missing Ratio' :df_na})
    return missing_data

In [5]:
train_miss = check_nmiss(train)
train_miss

,Missing Ratio
Employment.Type,3.285811


In [6]:
test_miss = check_nmiss(test)
test_miss

,Missing Ratio
Employment.Type,3.063385


Thus, there are no missing values other than emplyment type that needs to be treated

In [7]:
def prep_data_1(df):
    df['dob_date'] = pd.to_datetime(df['Date.of.Birth'])
    df['age']= pd.to_datetime("now") - df['dob_date']
    df['age'] = df['age'].dt.days
    df['disbursal_date'] = pd.to_datetime(df['DisbursalDate'])
    df['days_since_disbursal']= pd.to_datetime("now") - df['disbursal_date']
    df['days_since_disbursal'] = df['days_since_disbursal'].dt.days
    df['tot_ch_years'] = df['CREDIT.HISTORY.LENGTH'].str.extract('^([^yrs]*).*', expand=False)
    df['tot_ch_years'] = df['tot_ch_years'].astype('int')
    df['tot_ch_months_']=df['CREDIT.HISTORY.LENGTH'].str.split(' ').str[1]
    df['tot_ch_months'] = df['tot_ch_months_'].str.extract('^([^mon]*).*', expand=False)
    df['tot_ch_months'] = df['tot_ch_months'].astype('int')
    df['total_ch_months'] = df['tot_ch_years']*12 + df['tot_ch_months']
    df.drop(['tot_ch_years','tot_ch_months_','tot_ch_months'],axis=1, inplace=True)
    df['avg_acc_age_years'] = df['AVERAGE.ACCT.AGE'].str.extract('^([^yrs]*).*', expand=False)
    df['avg_acc_age_years'] = df['avg_acc_age_years'].astype('int')
    df['avg_acc_age_month_']=df['AVERAGE.ACCT.AGE'].str.split(' ').str[1]
    df['avg_acc_age_month'] = df['avg_acc_age_month_'].str.extract('^([^mon]*).*', expand=False)
    df['avg_acc_age_month'] = df['avg_acc_age_month'].astype('int')
    df['total_acc_age_months'] = df['avg_acc_age_years']*12 + df['avg_acc_age_month']
    df.drop(['avg_acc_age_years','avg_acc_age_month_','avg_acc_age_month'],axis=1, inplace=True)
    return df

In [8]:
train=prep_data_1(train)
test = prep_data_1(test)
train.shape,test.shape

((233154, 47), (112392, 46))

# Additional Features

In [9]:
def additional_feats(df):
    df['total_idproof_given']= df['Aadhar_flag']+df['PAN_flag']+df['VoterID_flag']+df['Driving_flag']+df['Passport_flag']
    df['pri_curr_balnc_pri_instal_ratio']=df['PRI.CURRENT.BALANCE']/df['PRIMARY.INSTAL.AMT']
    df['overdue_ac_to_activeAc_Ratio']=df['PRI.OVERDUE.ACCTS']/df['PRI.ACTIVE.ACCTS']
    df['Num_closed_loan_acc']=df['PRI.NO.OF.ACCTS']-df['PRI.OVERDUE.ACCTS']
    df['disbursed_to_asset_cost']= df['disbursed_amount']/df['asset_cost']
    df['ltv_diff_calc_ltv']=df['ltv']-df['disbursed_to_asset_cost']*100
    
    #df['avg_loan_tenure_to_credit_history_length']=df['total_acc_age_months']/df['total_ch_months']
    #df['avg_overdue_amt_per_outstanding_account']=df['PRI.CURRENT.BALANCE']/df['PRI.OVERDUE.ACCTS']
    #df['avg_num_default_accounts']=df['PRI.OVERDUE.ACCTS']/df['PRI.NO.OF.ACCTS']
    #df['total_deafult_acc_last_six_month_dlnqt']=df['PRI.OVERDUE.ACCTS']/df['DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS']
    ##
    #df['loans_passed_per_enquiry']=(df['PRI.NO.OF.ACCTS']+df['SEC.NO.OF.ACCTS'])/df['NO.OF_INQUIRIES']
    #df['Credit_score_X_hist_lnth_X_total_ac_age']=df['PERFORM_CNS.SCORE']*df['total_ch_months']*df['total_acc_age_months']
    #df['pct_dlnqut_loan_taken_last_6month']=df['DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS']/df['NEW.ACCTS.IN.LAST.SIX.MONTHS']
    #df['disbursal_penalized_by_cns_score']=df['disbursed_amount']/df['PERFORM_CNS.SCORE']
    #######
    #df['closed_loan_Ratio']=df['Num_closed_loan_acc']/df['PRI.NO.OF.ACCTS']
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    return df

In [10]:
train= additional_feats(train)
test = additional_feats(test)
train.shape,test.shape

((233154, 53), (112392, 52))

In [11]:
train['employ_type_salaried'] = np.where(train['Employment.Type']=='Salaried',1,0)
train['employ_type_selfemplyd'] = np.where(train['Employment.Type']=='Self employed',1,0)
test['employ_type_salaried'] = np.where(test['Employment.Type']=='Salaried',1,0)
test['employ_type_selfemplyd'] = np.where(test['Employment.Type']=='Self employed',1,0)
#train['employ_type_nan'] = np.where(train['Employment.Type']==np.nan,1,0)
#test['employ_type_nan'] = np.where(test['Employment.Type']==np.nan,1,0)

In [12]:
train.shape,test.shape

((233154, 55), (112392, 54))

# Non mean encoding model

In [13]:
train_columns = [c for c in train.columns if c not in ['UniqueID', 'loan_default','dob_date',
                                                       'disbursal_date','Date.of.Birth',
                                                       'Employment.Type','DisbursalDate','PERFORM_CNS.SCORE.DESCRIPTION',
                                                       'AVERAGE.ACCT.AGE','CREDIT.HISTORY.LENGTH',
                                                      'PERFORM_CNS.SCORE.DESCRIPTION',
                                                      'PERFORM_CNS.SCORE.DESCRIPTION',
                                                      'Credit_score_X_hist_lnth_X_total_ac_age']]
target = train['loan_default']
test_df = test[train_columns]

In [14]:
train.head()

,UniqueID,disbursed_amount,asset_cost,ltv,branch_id,supplier_id,manufacturer_id,Current_pincode_ID,Date.of.Birth,Employment.Type,...,total_ch_months,total_acc_age_months,total_idproof_given,pri_curr_balnc_pri_instal_ratio,overdue_ac_to_activeAc_Ratio,Num_closed_loan_acc,disbursed_to_asset_cost,ltv_diff_calc_ltv,employ_type_salaried,employ_type_selfemplyd
0,420825,50578,58400,89.55,67,22807,45,1441,01-01-84,Salaried,...,0,0,1,NaN,NaN,0,0.866062,2.943836,1,0
1,537409,47145,65550,73.23,67,22807,45,1502,31-07-85,Self employed,...,23,23,1,13.862381,1.0,0,0.719222,1.307803,0,1
2,417566,53278,61360,89.63,67,22807,45,1497,24-08-85,Self employed,...,0,0,1,NaN,NaN,0,0.868286,2.801447,0,1
3,624493,57513,66113,88.48,67,22807,45,1501,30-12-93,Self employed,...,15,8,1,0.000000,NaN,3,0.869920,1.488032,0,1
4,539055,52378,60300,88.39,67,22807,45,1495,09-12-77,Self employed,...,0,0,1,NaN,NaN,0,0.868624,1.527645,0,1


In [ ]:
##Best params till now
param = {'num_leaves': 65,
         'min_sum_hessian_in_leaf': 20.0,
         'min_data_in_leaf': 80, 
         'objective':'binary',
         'tree_learner': 'feature',   
         'boost_from_average':'true',
         'max_depth': -1,
         'learning_rate': 0.004,
         "min_child_samples": 50,
         "boosting": "gbdt",
         "feature_fraction": 0.15,
         "bagging_freq": 5,
         "bagging_fraction": 0.7 ,
         "bagging_seed": 11,
         "metric": 'auc',
         'reg_alpha': 0.35, 
         'reg_lambda': 0.25,
         "verbosity": -1,
         #categorical_feature': 'auto',
         'is_unbalanced':'True',
         "nthread": 6,
         "random_state": 99999}

In [22]:
param = {'num_leaves': 65,
         'min_sum_hessian_in_leaf': 20.0,
         'min_data_in_leaf': 80, 
         'objective':'binary',
         'tree_learner': 'feature',   
         'boost_from_average':'true',
         'max_depth': -1,
         'learning_rate': 0.004,
         "min_child_samples": 50,
         "boosting": "gbdt",
         "feature_fraction": 0.15,
         "bagging_freq": 5,
         "bagging_fraction": 0.7 ,
         "bagging_seed": 11,
         "metric": 'auc',
         'reg_alpha': 0.35, 
         'reg_lambda': 0.25,
         "verbosity": -1,
         #categorical_feature': 'auto',
         'is_unbalanced':'True',
         "nthread": 6,
         "random_state": 99999}
folds = StratifiedKFold(n_splits=7,shuffle=True, random_state=99999)
oof = np.zeros(len(train))
predictions = np.zeros(len(test_df))
feature_importance_df = pd.DataFrame()

for fold_, (trn_idx, val_idx) in enumerate(folds.split(train,train['loan_default'].values)):
    print("fold {}".format(fold_))
    trn_data = lgb.Dataset(train.iloc[trn_idx][train_columns], label=target.iloc[trn_idx])#, categorical_feature=categorical_feats)
    val_data = lgb.Dataset(train.iloc[val_idx][train_columns], label=target.iloc[val_idx])#, categorical_feature=categorical_feats)

    num_round = 100000
    clf = lgb.train(param, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=500, early_stopping_rounds = 1000)
    oof[val_idx] = clf.predict(train.iloc[val_idx][train_columns], num_iteration=clf.best_iteration)
    
    fold_importance_df = pd.DataFrame()
    fold_importance_df["Feature"] = train_columns
    fold_importance_df["importance"] = clf.feature_importance()
    fold_importance_df["fold"] = fold_ + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    
    predictions += clf.predict(test_df[train_columns], num_iteration=clf.best_iteration) / folds.n_splits

print("CV score: {:<8.5f}".format(roc_auc_score(target, oof)))

fold 0
Training until validation scores don't improve for 1000 rounds.
[500]	training's auc: 0.683132	valid_1's auc: 0.664418
[1000]	training's auc: 0.692293	valid_1's auc: 0.668443
[1500]	training's auc: 0.700855	valid_1's auc: 0.671648
[2000]	training's auc: 0.708319	valid_1's auc: 0.673537
[2500]	training's auc: 0.715057	valid_1's auc: 0.674858
[3000]	training's auc: 0.721309	valid_1's auc: 0.675831
[3500]	training's auc: 0.727261	valid_1's auc: 0.676662
[4000]	training's auc: 0.73296	valid_1's auc: 0.677227
[4500]	training's auc: 0.738382	valid_1's auc: 0.67762
[5000]	training's auc: 0.743413	valid_1's auc: 0.677955
[5500]	training's auc: 0.748289	valid_1's auc: 0.67825
[6000]	training's auc: 0.75307	valid_1's auc: 0.678423
[6500]	training's auc: 0.757648	valid_1's auc: 0.678646
[7000]	training's auc: 0.762083	valid_1's auc: 0.678716
[7500]	training's auc: 0.766363	valid_1's auc: 0.678779
[8000]	training's auc: 0.77051	valid_1's auc: 0.678806
[8500]	training's auc: 0.774568	valid_1

In [18]:
predictions

array([0.23452498, 0.2587706 , 0.22961304, ..., 0.1539375 , 0.22617216,
       0.10172662])

In [19]:
sub = pd.DataFrame({"UniqueID": test.UniqueID.values})
sub["loan_default"] = predictions
sub.to_csv(outloc + "sub_lightgbmCVscore_0.67656_7fold_lr0.0005_4192019_onlyfewadditionalvars.csv", index=False)

In [59]:
sub = pd.DataFrame({"UniqueID": test.UniqueID.values})
sub["loan_default"] = predictions
sub.to_csv(outloc + "sub_lightgbmCVscore_0.67905_910PM_lr001_7fold.csv", index=False)

In [23]:
train.shape

(233154, 55)

# k-fold Neural net

In [24]:
from sklearn.preprocessing import StandardScaler
##Keras specific
import keras
import keras.backend as K
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Dropout, BatchNormalization
from keras.optimizers import Adadelta
from keras import regularizers
from keras.wrappers.scikit_learn import KerasClassifier
from keras import optimizers

Using TensorFlow backend.


In [45]:

K.clear_session()
keras.backend.clear_session()
from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(2)
def ltfs_nn_model(df):
    nn_model = Sequential()
    nn_model.add(Dense(45, input_dim=45, kernel_initializer='normal', activation='relu'))
    nn_model.add(Dropout(0.5))
    nn_model.add(Dense(45, kernel_initializer='normal', activation='relu'))
    nn_model.add(Dropout(0.5))
    nn_model.add(Dense(30, kernel_initializer='normal', activation='relu'))
    nn_model.add(Dropout(0.5))
    nn_model.add(Dense(20, kernel_initializer='normal', activation='relu'))
    nn_model.add(Dropout(0.5))
    nn_model.add(Dense(5, kernel_initializer='normal', activation='relu'))
    nn_model.add(Dropout(0.5))
    nn_model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
    
    return nn_model
# Compile model
sgd = optimizers.SGD(lr=0.0001, decay=1e-6, momentum=0.9, nesterov=True)
adam = optimizers.Adam(lr=0.0001)

In [26]:
from sklearn.metrics import roc_auc_score
from keras.callbacks import Callback
class roc_callback(Callback):
    def __init__(self,training_data,validation_data):
        self.x = training_data[0]
        self.y = training_data[1]
        self.x_val = validation_data[0]
        self.y_val = validation_data[1]


    def on_train_begin(self, logs={}):
        return

    def on_train_end(self, logs={}):
        return

    def on_epoch_begin(self, epoch, logs={}):
        return

    def on_epoch_end(self, epoch, logs={}):
        y_pred = self.model.predict(self.x)
        roc = roc_auc_score(self.y, y_pred)
        y_pred_val = self.model.predict(self.x_val)
        roc_val = roc_auc_score(self.y_val, y_pred_val)
        print('\rroc-auc: %s - roc-auc_val: %s' % (str(round(roc,4)),str(round(roc_val,4))),end=100*' '+'\n')
        return

    def on_batch_begin(self, batch, logs={}):
        return

    def on_batch_end(self, batch, logs={}):
        return

In [46]:
train_columns = [c for c in train.columns if c not in ['UniqueID', 'loan_default','dob_date',
                                                       'disbursal_date','Date.of.Birth',
                                                       'Employment.Type','DisbursalDate','PERFORM_CNS.SCORE.DESCRIPTION',
                                                       'AVERAGE.ACCT.AGE','CREDIT.HISTORY.LENGTH',
                                                      'PERFORM_CNS.SCORE.DESCRIPTION',
                                                      'PERFORM_CNS.SCORE.DESCRIPTION',
                                                      'Credit_score_X_hist_lnth_X_total_ac_age']]
target = train['loan_default']
test_df = test[train_columns]
folds = StratifiedKFold(n_splits=5,shuffle=True, random_state=99999)
oof = np.zeros(len(train))
#predictions_nn = np.zeros(len(test_df))
feature_importance_df = pd.DataFrame()
all_cols = train.columns.tolist()
scaler = StandardScaler()

for fold_, (trn_idx, val_idx) in enumerate(folds.split(train,train['loan_default'].values)):
    print("fold {}".format(fold_))
    train_set = train.iloc[trn_idx][all_cols]
    val_set = train.iloc[val_idx][all_cols]
    test_df = test[train_columns]
    
    train_set.fillna(0,inplace=True)
    val_set.fillna(0,inplace=True)
    
    test_df.fillna(0,inplace=True)
    
    X_train,y_train= train_set[train_columns],train_set['loan_default']
    X_train = scaler.fit_transform(X_train)
    del train_set
    gc.collect()
    X_val,y_val = val_set[train_columns],val_set['loan_default']
    X_val = scaler.fit_transform(X_val)
    del val_set
    gc.collect()
    
    test_df_std = test_df[train_columns]
    tst_df_shape = test_df_std.shape[0]
    test_df_std = scaler.fit_transform(test_df_std)
    
    
    nn_model = ltfs_nn_model(X_train)
    
    
    nn_model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
    
    nn_model.fit(X_train, y_train, epochs=200, batch_size=400, verbose=1,validation_data=(X_val,y_val), callbacks=[roc_callback(training_data=(X_train, y_train),validation_data=(X_val,y_val))])
    
      
    
    oof = nn_model.predict(X_val)
    print("CV score Current Fold: {:<8.5f}".format(roc_auc_score(y_val, oof)))
    if fold_ == 0:
        predictions_nn = nn_model.predict(test_df_std) 
    elif fold_ >0:
        predictions_nn += nn_model.predict(test_df_std) / folds.n_splits



fold 0
Train on 186522 samples, validate on 46632 samples
Epoch 1/200
186522/186522 [==============================] - 3s 14us/step - loss: 0.6479 - acc: 0.7821 - val_loss: 0.5213 - val_acc: 0.7829
roc-auc: 0.6147 - roc-auc_val: 0.6105                                                                                                    
Epoch 2/200
186522/186522 [==============================] - 2s 10us/step - loss: 0.5573 - acc: 0.7829 - val_loss: 0.5150 - val_acc: 0.7829
roc-auc: 0.6268 - roc-auc_val: 0.6242                                                                                                    
Epoch 3/200
186522/186522 [==============================] - 2s 9us/step - loss: 0.5530 - acc: 0.7829 - val_loss: 0.5133 - val_acc: 0.7829
roc-auc: 0.6297 - roc-auc_val: 0.6275                                                                                                    
Epoch 4/200
186522/186522 [==============================] - 2s 9us/step - loss: 0.5489 - acc: 0.7829 - val_l

186522/186522 [==============================] - 3s 14us/step - loss: 0.5137 - acc: 0.7829 - val_loss: 0.5080 - val_acc: 0.7829
roc-auc: 0.6364 - roc-auc_val: 0.6331                                                                                                    
Epoch 31/200
186522/186522 [==============================] - 2s 13us/step - loss: 0.5139 - acc: 0.7829 - val_loss: 0.5081 - val_acc: 0.7829
roc-auc: 0.6363 - roc-auc_val: 0.6329                                                                                                    
Epoch 32/200
186522/186522 [==============================] - 3s 16us/step - loss: 0.5136 - acc: 0.7829 - val_loss: 0.5078 - val_acc: 0.7829
roc-auc: 0.6366 - roc-auc_val: 0.6332                                                                                                    
Epoch 33/200
186522/186522 [==============================] - 2s 12us/step - loss: 0.5135 - acc: 0.7829 - val_loss: 0.5079 - val_acc: 0.7829
roc-auc: 0.6365 - roc-auc_val: 0.63

roc-auc: 0.6377 - roc-auc_val: 0.6342                                                                                                    
Epoch 60/200
186522/186522 [==============================] - 3s 16us/step - loss: 0.5118 - acc: 0.7829 - val_loss: 0.5077 - val_acc: 0.7829
roc-auc: 0.6377 - roc-auc_val: 0.6343                                                                                                    
Epoch 61/200
186522/186522 [==============================] - 3s 14us/step - loss: 0.5122 - acc: 0.7829 - val_loss: 0.5076 - val_acc: 0.7829
roc-auc: 0.6378 - roc-auc_val: 0.6345                                                                                                    
Epoch 62/200
186522/186522 [==============================] - 2s 13us/step - loss: 0.5120 - acc: 0.7829 - val_loss: 0.5078 - val_acc: 0.7829
roc-auc: 0.638 - roc-auc_val: 0.6344                                                                                                    
Epoch 63/200
186522/186522

186522/186522 [==============================] - 2s 11us/step - loss: 0.5119 - acc: 0.7829 - val_loss: 0.5079 - val_acc: 0.7829
roc-auc: 0.6391 - roc-auc_val: 0.6351                                                                                                    
Epoch 90/200
186522/186522 [==============================] - 2s 12us/step - loss: 0.5116 - acc: 0.7829 - val_loss: 0.5073 - val_acc: 0.7829
roc-auc: 0.6392 - roc-auc_val: 0.6353                                                                                                    
Epoch 91/200
186522/186522 [==============================] - 3s 14us/step - loss: 0.5115 - acc: 0.7829 - val_loss: 0.5072 - val_acc: 0.7829
roc-auc: 0.6391 - roc-auc_val: 0.6351                                                                                                    
Epoch 92/200
186522/186522 [==============================] - 3s 14us/step - loss: 0.5121 - acc: 0.7829 - val_loss: 0.5076 - val_acc: 0.7829
roc-auc: 0.639 - roc-auc_val: 0.634

roc-auc: 0.6397 - roc-auc_val: 0.6349                                                                                                    
Epoch 119/200
186522/186522 [==============================] - 2s 13us/step - loss: 0.5116 - acc: 0.7829 - val_loss: 0.5076 - val_acc: 0.7829
roc-auc: 0.6398 - roc-auc_val: 0.6348                                                                                                    
Epoch 120/200
186522/186522 [==============================] - 2s 13us/step - loss: 0.5115 - acc: 0.7829 - val_loss: 0.5075 - val_acc: 0.7829
roc-auc: 0.6399 - roc-auc_val: 0.6347                                                                                                    
Epoch 121/200
186522/186522 [==============================] - 3s 16us/step - loss: 0.5113 - acc: 0.7829 - val_loss: 0.5074 - val_acc: 0.7829
roc-auc: 0.6399 - roc-auc_val: 0.6348                                                                                                    
Epoch 122/200
186522/1

...

186524/186524 [==============================] - 3s 16us/step - loss: 0.5116 - acc: 0.7829 - val_loss: 0.5077 - val_acc: 0.7829
roc-auc: 0.6399 - roc-auc_val: 0.6313                                                                                                    
Epoch 81/200
186524/186524 [==============================] - 3s 16us/step - loss: 0.5114 - acc: 0.7829 - val_loss: 0.5079 - val_acc: 0.7829
roc-auc: 0.6396 - roc-auc_val: 0.6314                                                                                                    
Epoch 82/200
186524/186524 [==============================] - 3s 14us/step - loss: 0.5113 - acc: 0.7829 - val_loss: 0.5080 - val_acc: 0.7829
roc-auc: 0.6396 - roc-auc_val: 0.6312                                                                                                    
Epoch 83/200
186524/186524 [==============================] - 3s 16us/step - loss: 0.5114 - acc: 0.7829 - val_loss: 0.5079 - val_acc: 0.7829
roc-auc: 0.6397 - roc-auc_val: 0.63

roc-auc: 0.6411 - roc-auc_val: 0.6316                                                                                                    
Epoch 110/200
186524/186524 [==============================] - 3s 15us/step - loss: 0.5113 - acc: 0.7829 - val_loss: 0.5079 - val_acc: 0.7829
roc-auc: 0.6411 - roc-auc_val: 0.6316                                                                                                    
Epoch 111/200
186524/186524 [==============================] - 3s 18us/step - loss: 0.5115 - acc: 0.7829 - val_loss: 0.5078 - val_acc: 0.7829
roc-auc: 0.6409 - roc-auc_val: 0.6316                                                                                                    
Epoch 112/200
186524/186524 [==============================] - 3s 15us/step - loss: 0.5106 - acc: 0.7829 - val_loss: 0.5080 - val_acc: 0.7829
roc-auc: 0.6411 - roc-auc_val: 0.6315                                                                                                    
Epoch 113/200
186524/1

186524/186524 [==============================] - 2s 12us/step - loss: 0.5107 - acc: 0.7829 - val_loss: 0.5081 - val_acc: 0.7829
roc-auc: 0.6421 - roc-auc_val: 0.6315                                                                                                    
Epoch 140/200
186524/186524 [==============================] - 4s 19us/step - loss: 0.5107 - acc: 0.7829 - val_loss: 0.5080 - val_acc: 0.7829
roc-auc: 0.6422 - roc-auc_val: 0.6316                                                                                                    
Epoch 141/200
186524/186524 [==============================] - 4s 22us/step - loss: 0.5108 - acc: 0.7829 - val_loss: 0.5081 - val_acc: 0.7829
roc-auc: 0.6422 - roc-auc_val: 0.6314                                                                                                    
Epoch 142/200
186524/186524 [==============================] - 3s 14us/step - loss: 0.5108 - acc: 0.7829 - val_loss: 0.5081 - val_acc: 0.7829
roc-auc: 0.6422 - roc-auc_val: 0

186524/186524 [==============================] - 2s 12us/step - loss: 0.5108 - acc: 0.7829 - val_loss: 0.5081 - val_acc: 0.7829
roc-auc: 0.6433 - roc-auc_val: 0.6321                                                                                                    
Epoch 169/200
186524/186524 [==============================] - 2s 9us/step - loss: 0.5110 - acc: 0.7829 - val_loss: 0.5081 - val_acc: 0.7829
roc-auc: 0.6434 - roc-auc_val: 0.6326                                                                                                    
Epoch 170/200
186524/186524 [==============================] - 3s 17us/step - loss: 0.5107 - acc: 0.7829 - val_loss: 0.5080 - val_acc: 0.7829
roc-auc: 0.6436 - roc-auc_val: 0.6326                                                                                                    
Epoch 171/200
186524/186524 [==============================] - 2s 12us/step - loss: 0.5107 - acc: 0.7829 - val_loss: 0.5080 - val_acc: 0.7829
roc-auc: 0.6434 - roc-auc_val: 0.

186524/186524 [==============================] - 2s 10us/step - loss: 0.5111 - acc: 0.7829 - val_loss: 0.5081 - val_acc: 0.7829
roc-auc: 0.6438 - roc-auc_val: 0.6327                                                                                                    
Epoch 198/200
186524/186524 [==============================] - 3s 15us/step - loss: 0.5106 - acc: 0.7829 - val_loss: 0.5082 - val_acc: 0.7829
roc-auc: 0.6438 - roc-auc_val: 0.6325                                                                                                    
Epoch 199/200
186524/186524 [==============================] - 2s 11us/step - loss: 0.5105 - acc: 0.7829 - val_loss: 0.5082 - val_acc: 0.7829
roc-auc: 0.6439 - roc-auc_val: 0.6324                                                                                                    
Epoch 200/200
186524/186524 [==============================] - 2s 12us/step - loss: 0.5108 - acc: 0.7829 - val_loss: 0.5081 - val_acc: 0.7829
roc-auc: 0.6442 - roc-auc_val: 0

In [47]:
predictions_nn

array([[0.4538212 ],
       [0.48323599],
       [0.4360031 ],
       ...,
       [0.33332962],
       [0.4608469 ],
       [0.27828225]], dtype=float32)

In [48]:
sub = pd.DataFrame({"UniqueID": test.UniqueID.values})
sub["loan_default"] = predictions_nn
sub.to_csv(outloc + "fianl_submission_NN_04212019.csv", index=False)